In [ ]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from link_prediction import *

In [2]:
#Load the entity embeddings dict
with open("wn18rr_transe_entity_embeddings_2.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("wn18rr_transe_predicate_embeddings_2.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

train_triples = pd.read_csv('wn18rr_train.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])
test_triples = pd.read_csv('wn18rr_test.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])

**Create Negative Samples**

In [3]:
training_object=Training(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE')
tail_df = training_object.create_training_data_filtered(n=1, creating_for="tail")
head_df = training_object.create_training_data_filtered(n=1, creating_for="head")
print(len(tail_df))
print(len(head_df))

Processing triples: 100%|██████████| 86835/86835 [02:00<00:00, 723.04triple/s]


62547
40962


**Prepare Training Data**

In [4]:
train_triples['label']=1
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: training_object.get_embedding(row).cpu().numpy(), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)
train_df['label'].value_counts()

,count
label,
0,94479
1,86835


**Set Models**

**MLP**

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [6]:
param_grid_mlp = {
    'hidden_layer_sizes': [ (64, 64), (128, 128), (256, 256)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01],
    'max_iter': [50, 100]
}

results_mlp = []
for hidden_layer_sizes in param_grid_mlp['hidden_layer_sizes']:
    for alpha in param_grid_mlp['alpha']:
        for learning_rate_init in param_grid_mlp['learning_rate_init']:
            for max_iter in param_grid_mlp['max_iter']:
                mlp = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    alpha=alpha,
                    learning_rate_init=learning_rate_init,
                    max_iter=max_iter,
                    solver='adam',
                    early_stopping=True,
                    batch_size=32,
                    random_state=42
                )
                mlp.fit(X_train, y_train)
                preds = mlp.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_mlp.append({
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'alpha': alpha,
                    'learning_rate_init': learning_rate_init,
                    'max_iter': max_iter,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_mlp = pd.DataFrame(results_mlp)

best_mlp_params = df_results_mlp.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for MLP:", best_mlp_params)

Best parameters for MLP: {'hidden_layer_sizes': (128, 128), 'alpha': 0.01, 'learning_rate_init': 0.001, 'max_iter': 100, 'f1_score': 0.850262599173092, 'accuracy': 0.8521950143392897, 'precision_0': 0.8794572774164611, 'precision_1': 0.8258085522031691, 'recall_0': 0.8301227773073666, 'recall_1': 0.8762091202210963}


**XGB**

In [7]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    # 'scale_pos_weight': [0.5, 1, 2],  # commented out as provided
    'learning_rate': [0.01, 0.1, 0.2]
}


results_xgb = []
for n_estimators in param_grid_xgb['n_estimators']:
    for max_depth in param_grid_xgb['max_depth']:
        for learning_rate in param_grid_xgb['learning_rate']:
            xgb = XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                random_state=42,
            )
            xgb.fit(X_train, y_train)
            preds = xgb.predict(X_test)

            # Calculate metrics
            score_f1 = f1_score(y_test, preds)
            acc = accuracy_score(y_test, preds)
            precisions = precision_score(y_test, preds, average=None, zero_division=0)
            recalls = recall_score(y_test, preds, average=None, zero_division=0)

            results_xgb.append({
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'f1_score': score_f1,
                'accuracy': acc,
                'precision_0': precisions[0],
                'precision_1': precisions[1],
                'recall_0': recalls[0],
                'recall_1': recalls[1]
            })

df_results_xgb = pd.DataFrame(results_xgb)
best_xgb_params = df_results_xgb.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for XGBClassifier:", best_xgb_params)

Best parameters for XGBClassifier: {'n_estimators': 300.0, 'max_depth': 5.0, 'learning_rate': 0.2, 'f1_score': 0.9000225682690137, 'accuracy': 0.9022722258989632, 'precision_0': 0.9221293444786626, 'precision_1': 0.8823008849557522, 'recall_0': 0.8873835732430144, 'recall_1': 0.9184707508060801}


**LGBM**

In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2],
    'is_unbalance': [True, False]
}

results_lgbm = []
for n_estimators in param_grid_lgbm['n_estimators']:
    for max_depth in param_grid_lgbm['max_depth']:
        for learning_rate in param_grid_lgbm['learning_rate']:
            for is_unbalance in param_grid_lgbm['is_unbalance']:
                lgbm = LGBMClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    is_unbalance=is_unbalance,
                    random_state=42
                )
                lgbm.fit(X_train, y_train)
                preds = lgbm.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_lgbm.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'is_unbalance': is_unbalance,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_lgbm = pd.DataFrame(results_lgbm)
best_lgbm_params = df_results_lgbm.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for LGBM:", best_lgbm_params)

**Train Ensemble**

In [9]:
# Best MLPClassifier
best_mlp = MLPClassifier(
    hidden_layer_sizes=best_mlp_params['hidden_layer_sizes'],
    alpha=best_mlp_params['alpha'],
    learning_rate_init=best_mlp_params['learning_rate_init'],
    max_iter=best_mlp_params['max_iter'],
    solver='adam',
    early_stopping=True,
    batch_size=32,
    random_state=42
)
best_mlp.fit(X_train, y_train)
preds_mlp = best_mlp.predict(X_test)
report_mlp = classification_report(y_test, preds_mlp)

# Best XGBClassifier
best_xgb = XGBClassifier(
    n_estimators=int(best_xgb_params['n_estimators']),
    max_depth=int(best_xgb_params['max_depth']),
    learning_rate=float(best_xgb_params['learning_rate']),
    random_state=42,
)
best_xgb.fit(X_train, y_train)
preds_xgb = best_xgb.predict(X_test)
report_xgb = classification_report(y_test, preds_xgb)

# Best LGBMClassifier
best_lgbm = LGBMClassifier(
    n_estimators=best_lgbm_params['n_estimators'],
    max_depth=best_lgbm_params['max_depth'],
    learning_rate=best_lgbm_params['learning_rate'],
    is_unbalance=best_lgbm_params['is_unbalance'],
    random_state=42
)
best_lgbm.fit(X_train, y_train)
preds_lgbm = best_lgbm.predict(X_test)
report_lgbm = classification_report(y_test, preds_lgbm)

######################################
# Ensemble the best models using VotingClassifier
######################################
ensemble_model = VotingClassifier(
    estimators=[
        ('mlp', best_mlp),
        ('xgb', best_xgb),
        ('lgbm', best_lgbm)
    ],
    voting='soft',  # Soft voting uses predicted probabilities
    n_jobs=-1
)

start_time = time.time()
ensemble_model.fit(X_train, y_train)
end_time = time.time()

ensemble_time = end_time - start_time
preds_ensemble = ensemble_model.predict(X_test)
ensemble_acc = accuracy_score(y_test, preds_ensemble)
report_ensemble = classification_report(y_test, preds_ensemble)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 78151, number of negative: 85031
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.191478 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 52199
[LightGBM] [Info] Number of data points in the train set: 163182, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.478919 -> initscore=-0.084373
[LightGBM] [Info] Start training from score -0.084373


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [10]:
y_pred_ensemble = ensemble_model.predict(X_test)
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Ensemble Classification Report:\n", classification_report(y_test, y_pred_ensemble))

Ensemble Accuracy: 0.9144606221045665
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.90      0.92      9448
           1       0.89      0.94      0.91      8684

    accuracy                           0.91     18132
   macro avg       0.91      0.92      0.91     18132
weighted avg       0.92      0.91      0.91     18132



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
with open("/content/drive/MyDrive/link_prediction/tuning/wn18rr_transe_tuning.txt", "w") as f:
    f.write("##############################\n")
    f.write("MLPClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_mlp.to_string())
    f.write("\n\nBest Parameters for MLPClassifier:\n")
    f.write(str(best_mlp_params))
    f.write("\n\nClassification Report for Best MLPClassifier:\n")
    f.write(report_mlp)

    f.write("\n\n\n##############################\n")
    f.write("XGBClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_xgb.to_string())
    f.write("\n\nBest Parameters for XGBClassifier:\n")
    f.write(str(best_xgb_params))
    f.write("\n\nClassification Report for Best XGBClassifier:\n")
    f.write(report_xgb)

    f.write("\n\n\n##############################\n")
    f.write("LGBMClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_lgbm.to_string())
    f.write("\n\nBest Parameters for LGBMClassifier:\n")
    f.write(str(best_lgbm_params))
    f.write("\n\nClassification Report for Best LGBMClassifier:\n")
    f.write(report_lgbm)

    f.write("\n\n\n##############################\n")
    f.write("Ensemble Model Performance\n")
    f.write("##############################\n")
    f.write("Ensemble Accuracy: {:.4f}\n".format(ensemble_acc))
    f.write("Ensemble Classification Report:\n")
    f.write(report_ensemble)

print("Results saved to model_results.txt")

Results saved to model_results.txt


**Rerank Hitk**

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE', k=50)

Precomputing top-k predictions: 100%|██████████| 3134/3134 [00:41<00:00, 74.68it/s]


In [14]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10,20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19,20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 0.90% | tail: 1.10% | overall: 1.01%
k=1 | th=2 | head: 1.57% | tail: 2.31% | overall: 1.96%
k=1 | th=3 | head: 1.89% | tail: 2.49% | overall: 2.20%
k=1 | th=4 | head: 2.28% | tail: 2.84% | overall: 2.58%
k=1 | th=5 | head: 2.20% | tail: 2.70% | overall: 2.46%
k=1 | th=6 | head: 2.20% | tail: 2.63% | overall: 2.43%
k=1 | th=7 | head: 2.08% | tail: 2.35% | overall: 2.22%
k=1 | th=8 | head: 2.08% | tail: 2.35% | overall: 2.22%
k=1 | th=9 | head: 1.97% | tail: 2.31% | overall: 2.15%
k=1 | th=10 | head: 1.89% | tail: 2.24% | overall: 2.07%
k=1 | th=20 | head: 1.30% | tail: 1.67% | overall: 1.49%
k=1 | th=30 | head: 1.22% | tail: 1.21% | overall: 1.21%
k=1 | th=40 | head: 1.22% | tail: 1.17% | overall: 1.19%
k=1 | th=50 | head: 1.10% | tail: 1.17% | overall: 1.14%
k=3 | th=3 | head: 38.67% | tail: 36.53% | overall: 37.55%
k=3 | th=4 | head: 39.89% | tail: 37.67% | overall: 38.72%
k=3 | th=5 | head: 39.61% | tail: 37.81% | overall: 38.67%
k=3 | th=6 | head: 38.59% | tail: 

In [15]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 0.90% | tail: 1.10% | overall: 1.01%
k=1 | th=2 | head: 1.06% | tail: 2.27% | overall: 1.70%
k=1 | th=3 | head: 1.18% | tail: 2.59% | overall: 1.92%
k=1 | th=4 | head: 1.34% | tail: 2.84% | overall: 2.13%
k=1 | th=5 | head: 1.30% | tail: 2.91% | overall: 2.15%
k=1 | th=6 | head: 1.30% | tail: 2.81% | overall: 2.09%
k=1 | th=7 | head: 1.18% | tail: 2.88% | overall: 2.07%
k=1 | th=8 | head: 1.18% | tail: 3.02% | overall: 2.15%
k=1 | th=9 | head: 1.10% | tail: 3.20% | overall: 2.20%
k=1 | th=10 | head: 1.02% | tail: 3.20% | overall: 2.17%
k=1 | th=20 | head: 0.98% | tail: 2.84% | overall: 1.96%
k=1 | th=30 | head: 0.83% | tail: 2.24% | overall: 1.57%
k=1 | th=40 | head: 0.79% | tail: 2.03% | overall: 1.44%
k=1 | th=50 | head: 0.75% | tail: 1.92% | overall: 1.36%
k=3 | th=3 | head: 38.67% | tail: 36.53% | overall: 37.55%
k=3 | th=4 | head: 40.83% | tail: 38.49% | overall: 39.60%
k=3 | th=5 | head: 41.42% | tail: 38.98% | overall: 40.14%
k=3 | th=6 | head: 41.15% | tail: 

In [16]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 0.90% | tail: 1.10% | overall: 1.01%
k=1 | th=2 | head: 1.18% | tail: 2.38% | overall: 1.81%
k=1 | th=3 | head: 1.42% | tail: 2.81% | overall: 2.15%
k=1 | th=4 | head: 1.57% | tail: 3.09% | overall: 2.37%
k=1 | th=5 | head: 1.49% | tail: 3.23% | overall: 2.41%
k=1 | th=6 | head: 1.53% | tail: 3.13% | overall: 2.37%
k=1 | th=7 | head: 1.49% | tail: 3.09% | overall: 2.33%
k=1 | th=8 | head: 1.42% | tail: 3.23% | overall: 2.37%
k=1 | th=9 | head: 1.34% | tail: 3.41% | overall: 2.43%
k=1 | th=10 | head: 1.30% | tail: 3.38% | overall: 2.39%
k=1 | th=20 | head: 1.18% | tail: 2.95% | overall: 2.11%
k=1 | th=30 | head: 1.10% | tail: 2.59% | overall: 1.89%
k=1 | th=40 | head: 1.10% | tail: 2.35% | overall: 1.76%
k=1 | th=50 | head: 1.18% | tail: 2.06% | overall: 1.64%
k=3 | th=3 | head: 38.67% | tail: 36.53% | overall: 37.55%
k=3 | th=4 | head: 41.11% | tail: 38.56% | overall: 39.77%
k=3 | th=5 | head: 41.62% | tail: 39.37% | overall: 40.44%
k=3 | th=6 | head: 41.58% | tail: 

In [17]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 0.90% | tail: 1.10% | overall: 1.01%
k=1 | th=2 | head: 1.22% | tail: 2.31% | overall: 1.79%
k=1 | th=3 | head: 1.34% | tail: 2.59% | overall: 2.00%
k=1 | th=4 | head: 1.42% | tail: 2.91% | overall: 2.20%
k=1 | th=5 | head: 1.53% | tail: 2.99% | overall: 2.30%
k=1 | th=6 | head: 1.49% | tail: 2.77% | overall: 2.17%
k=1 | th=7 | head: 1.46% | tail: 2.70% | overall: 2.11%
k=1 | th=8 | head: 1.42% | tail: 2.88% | overall: 2.18%
k=1 | th=9 | head: 1.34% | tail: 3.13% | overall: 2.28%
k=1 | th=10 | head: 1.26% | tail: 3.02% | overall: 2.18%
k=1 | th=20 | head: 1.02% | tail: 2.74% | overall: 1.92%
k=1 | th=30 | head: 0.79% | tail: 2.35% | overall: 1.61%
k=1 | th=40 | head: 0.75% | tail: 1.95% | overall: 1.38%
k=1 | th=50 | head: 0.71% | tail: 1.81% | overall: 1.29%
k=3 | th=3 | head: 38.67% | tail: 36.53% | overall: 37.55%
k=3 | th=4 | head: 41.11% | tail: 38.63% | overall: 39.81%
k=3 | th=5 | head: 42.13% | tail: 39.34% | overall: 40.66%
k=3 | th=6 | head: 42.29% | tail: 